Local area migration indicators UK: Migration Flows

In [1]:
from databaker.framework import *
import pandas as pd

In [2]:
from pathlib import Path

sourceFolder = Path('in')
sourceFolder.mkdir(exist_ok=True)

%run lib/scrape_ons.ipynb

metadata = scrape('https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/' \
                  'migrationwithintheuk/datasets/localareamigrationindicatorsunitedkingdom')
metadata

{'about': 'Interactive product bringing together different migration-related data sources at local authority level.',
 'fileURL': 'https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/migrationwithintheuk/datasets/localareamigrationindicatorsunitedkingdom/current/publicviewmastercopylocalareamigrationindicatoraugust2017feb2018update.xls',
 'mailto': 'mailto:migstatsunit@ons.gsi.gov.uk',
 'nextRelease': datetime.date(2018, 8, 23),
 'releaseDate': datetime.date(2017, 8, 24),
 'title': 'Local Area Migration Indicators, UK'}

https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/migrationwithintheuk/datasets/localareamigrationindicatorsunitedkingdom/current/publicviewmastercopylocalareamigrationindicatoraugust2017feb2018update.xls

In [3]:
inputFile = sourceFolder / 'data.xls'
response = session.get(metadata['fileURL'])
with open(inputFile, 'wb') as f:
  f.write(response.content)
tab = loadxlstabs(inputFile, sheetids='Migration Flows')[0]

Loading in\data.xls which has size 3976704 bytes
Table names: ['Migration Flows']


In [4]:
observations = tab.excel_ref('C5').expand(DOWN).expand(RIGHT).is_not_blank()
observations

{<AA362 8251.0>, <C72 176587.0>, <AJ170 7754.0>, <H262 34934.0>, <AJ333 6148.0>, <AQ248 111243.0>, <I212 2206.0>, <I298 607.0>, <AT376 21205.0>, <H217 56519.0>, <AA292 5356.0>, <S282 953.0>, <T126 51.0>, <BF90 156837.0>, <AJ347 884.0>, <AA216 3607.0>, <X12 37688.0>, <AV398 367250.0>, <AB250 104120.0>, <BD115 4375.0>, <O382 3135.0>, <AJ143 4364.0>, <AR130 168.0>, <T25 2468.0>, <M351 283273.0>, <AK342 6302.0>, <AX122 97.0>, <BD259 5670.0>, <AN355 5494.0>, <AY200 5181.0>, <AV241 157267.0>, <N265 305.0>, <AG88 264885.0>, <K207 2826.0>, <AV191 105261.0>, <AS136 173.0>, <BH431 334.0>, <AL279 76052.0>, <J377 3331.0>, <N285 853.0>, <BH200 142.0>, <Y131 72.0>, <BI194 5458.0>, <G39 33915.0>, <N10 17175.0>, <AT287 7718.0>, <E182 564.0>, <AH333 1690.0>, <X355 9216.0>, <BD374 26806.0>, <Q31 44607.0>, <R310 81615.0>, <AW279 477.0>, <I264 728.0>, <BB237 289.0>, <AE289 2747.0>, <G356 17950.0>, <AP376 22715.0>, <BJ289 2926.0>, <N48 8103.0>, <AZ376 24017.0>, <G396 859.0>, <X132 171.0>, <AC256 738.0>, <M

In [5]:
MigrationArea = tab.excel_ref('A5').expand(DOWN).is_not_blank()
MigrationArea

{<A245 'E07000169'>, <A57 'E06000004'>, <A316 'E08000006'>, <A295 'E07000224'>, <A96 'E06000043'>, <A394 'S12000010'>, <A156 'E07000070'>, <A252 'E07000176'>, <A178 'E07000092'>, <A310 'E07000243'>, <A129 'E07000036'>, <A36 'E10000021'>, <A209 'E07000127'>, <A378 'E09000032'>, <A184 'E07000099'>, <A12 'E12000003'>, <A325 'E08000015'>, <A360 'E09000014'>, <A114 'E07000008'>, <A412 'S12000034'>, <A356 'E09000010'>, <A367 'E09000021'>, <A196 'E07000114'>, <A441 'W06000021'>, <A276 'E07000205'>, <A33 'E10000018'>, <A195 'E07000113'>, <A172 'E07000086'>, <A212 'E07000130'>, <A124 'E07000031'>, <A428 'W06000006'>, <A237 'E07000155'>, <A244 'E07000168'>, <A418 'S12000041'>, <A265 'E07000194'>, <A366 'E09000020'>, <A343 'E08000034'>, <A26 'E10000011'>, <A260 'E07000189'>, <A361 'E09000015'>, <A20 'E10000002'>, <A123 'E07000030'>, <A87 'E06000034'>, <A258 'E07000187'>, <A339 'E08000030'>, <A104 'E06000052'>, <A84 'E06000031'>, <A436 'W06000015'>, <A45 'E10000032'>, <A86 'E06000033'>, <A200 'E07

In [6]:
Period = tab.excel_ref('C1').expand(RIGHT).is_not_blank()
Period

{<R1 'Mid-2007 to Mid-2008 '>, <C1 'Mid-2004 to Mid-2005 '>, <AG1 'Mid-2010 to Mid-2011'>, <BF1 'Mid-2015 to Mid-2016'>, <AB1 'Mid-2009 to Mid-2010 '>, <W1 'Mid-2008 to Mid-2009 '>, <M1 'Mid-2006 to Mid-2007 '>, <AV1 'Mid-2013 to Mid-2014'>, <AQ1 'Mid-2012 to Mid-2013'>, <BA1 'Mid-2014 to Mid-2015'>, <H1 'Mid-2005 to Mid-2006'>, <AL1 'Mid-2011 to Mid-2012'>}

In [7]:
Migration = tab.excel_ref('C2').expand(RIGHT).is_not_blank()
Migration

{<BA2 'Mid-2015 Population Estimate'>, <P2 'Internal Migration\n (within UK)'>, <M2 'Mid-2007 Population Estimate'>, <K2 'Internal Migration\n (within UK)'>, <H2 'Mid-2006 Population Estimate'>, <BF2 'Mid-2016 Population Estimate'>, <AL2 'Mid-2012 Population Estimate'>, <AV2 'Mid-2014 Population Estimate'>, <I2 'Long-Term International Migration'>, <S2 'Long-Term International Migration'>, <AJ2 'Internal Migration\n (within UK)'>, <Z2 'Internal Migration\n (within UK)'>, <D2 'Long-Term International Migration'>, <AH2 'Long-Term International Migration'>, <AG2 'Mid-2011 Population Estimate'>, <AB2 'Mid-2010 Population Estimate'>, <BD2 'Internal Migration\n (within UK)'>, <AT2 'Internal Migration\n (within UK)'>, <R2 'Mid-2008 Population Estimate'>, <F2 'Internal Migration\n (within UK)'>, <AQ2 'Mid-2013 Population Estimate'>, <C2 'Mid-2005 Population Estimate'>, <N2 'Long-Term International Migration'>, <X2 'Long-Term International Migration'>, <AC2 'Long-Term International Migration'>,

In [8]:
Flow = tab.excel_ref('D3').expand(RIGHT).is_not_blank()
Flow

{<O3 'Outflow'>, <N3 'Inflow'>, <I3 'Inflow'>, <Q3 'Outflow'>, <BH3 'Outflow'>, <AJ3 'Inflow'>, <AU3 'Outflow'>, <BG3 'Inflow'>, <BB3 'Inflow'>, <AW3 'Inflow'>, <AF3 'Outflow'>, <Y3 'Outflow'>, <V3 'Outflow'>, <X3 'Inflow'>, <S3 'Inflow'>, <AT3 'Inflow'>, <AO3 'Inflow'>, <AR3 'Inflow'>, <AM3 'Inflow'>, <P3 'Inflow'>, <K3 'Inflow'>, <F3 'Inflow'>, <U3 'Inflow'>, <AN3 'Outflow'>, <AD3 'Outflow'>, <L3 'Outflow'>, <E3 'Outflow'>, <BE3 'Outflow'>, <BD3 'Inflow'>, <AY3 'Inflow'>, <AA3 'Outflow'>, <AC3 'Inflow'>, <AX3 'Outflow'>, <T3 'Outflow'>, <J3 'Outflow'>, <BJ3 'Outflow'>, <AK3 'Outflow'>, <AI3 'Outflow'>, <G3 'Outflow'>, <AH3 'Inflow'>, <BI3 'Inflow'>, <D3 'Inflow'>, <AE3 'Inflow'>, <Z3 'Inflow'>, <BC3 'Outflow'>, <AZ3 'Outflow'>, <AS3 'Outflow'>, <AP3 'Outflow'>}

In [9]:
Dimensions = [
            HDim(Period,'Period',CLOSEST,LEFT),
            HDim(MigrationArea,'Migration Area', DIRECTLY, LEFT),
            HDim(Migration,'Migration',CLOSEST,LEFT),
            HDim(Flow,'Flow',DIRECTLY,ABOVE),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People (thousands)')
            ]

In [10]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# savepreviewhtml(c1)

In [11]:
new_table = c1.topandas()
new_table

,OBS,Period,Migration Area,Migration,Flow,Measure Type,Unit
0,50606034.0,Mid-2004 to Mid-2005,E92000001,Mid-2005 Population Estimate,None,Count,People (thousands)
1,585266.0,Mid-2004 to Mid-2005,E92000001,Long-Term International Migration,Inflow,Count,People (thousands)
2,285583.0,Mid-2004 to Mid-2005,E92000001,Long-Term International Migration,Outflow,Count,People (thousands)
3,102457.0,Mid-2004 to Mid-2005,E92000001,Internal Migration\n (within UK),Inflow,Count,People (thousands)
4,123867.0,Mid-2004 to Mid-2005,E92000001,Internal Migration\n (within UK),Outflow,Count,People (thousands)
5,50965186.0,Mid-2005 to Mid-2006,E92000001,Mid-2006 Population Estimate,None,Count,People (thousands)
6,551187.0,Mid-2005 to Mid-2006,E92000001,Long-Term International Migration,Inflow,Count,People (thousands)
7,339983.0,Mid-2005 to Mid-2006,E92000001,Long-Term International Migration,Outflow,Count,People (thousands)
8,103144.0,Mid-2005 to Mid-2006,E92000001,Internal Migration\n (within UK),Inflow,Count,People (thousands)
9,119006.0,Mid-2005 to Mid-2006,E92000001,Internal Migration\n (within UK),Outflow,Count,People (thousands)


In [12]:
new_table.count()

OBS               26220
Period            26220
Migration Area    26220
Migration         26220
Flow              20976
Measure Type      26220
Unit              26220
dtype: int64

In [13]:
new_table.dropna(inplace=True)

In [14]:
new_table.head()

,OBS,Period,Migration Area,Migration,Flow,Measure Type,Unit
1,585266.0,Mid-2004 to Mid-2005,E92000001,Long-Term International Migration,Inflow,Count,People (thousands)
2,285583.0,Mid-2004 to Mid-2005,E92000001,Long-Term International Migration,Outflow,Count,People (thousands)
3,102457.0,Mid-2004 to Mid-2005,E92000001,Internal Migration\n (within UK),Inflow,Count,People (thousands)
4,123867.0,Mid-2004 to Mid-2005,E92000001,Internal Migration\n (within UK),Outflow,Count,People (thousands)
6,551187.0,Mid-2005 to Mid-2006,E92000001,Long-Term International Migration,Inflow,Count,People (thousands)


In [15]:
new_table['Value'] = new_table['OBS'].astype(int)

In [16]:
new_table = new_table[['Period','Migration Area','Migration','Flow','Measure Type','Value','Unit']]

In [17]:
new_table.head()

,Period,Migration Area,Migration,Flow,Measure Type,Value,Unit
1,Mid-2004 to Mid-2005,E92000001,Long-Term International Migration,Inflow,Count,585266,People (thousands)
2,Mid-2004 to Mid-2005,E92000001,Long-Term International Migration,Outflow,Count,285583,People (thousands)
3,Mid-2004 to Mid-2005,E92000001,Internal Migration\n (within UK),Inflow,Count,102457,People (thousands)
4,Mid-2004 to Mid-2005,E92000001,Internal Migration\n (within UK),Outflow,Count,123867,People (thousands)
6,Mid-2005 to Mid-2006,E92000001,Long-Term International Migration,Inflow,Count,551187,People (thousands)


In [18]:
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

new_table.to_csv(destinationFolder / ('migrationflows.csv'), index = False)

In [19]:
writeMetadata(metadata, 'Local Area Migration Indicators', 'Migration')

In [20]:
new_table.count()

Period            20976
Migration Area    20976
Migration         20976
Flow              20976
Measure Type      20976
Value             20976
Unit              20976
dtype: int64